In [2]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [40]:
import pandas as pd
from fuzzywuzzy import process, fuzz
import re, ast

# Load the CSV file containing the abbreviations
abbreviations_df = pd.read_csv('abbreviations.csv')

display(abbreviations_df)

tables = pd.read_csv('all_tables_to_annotate.csv')
display(tables)

def remove_consecutive_duplicates(lst):
    result = []
    prev_item = None
    for item in lst:
        if item != prev_item:
            result.append(item)
        prev_item = item
    return result
    
# Define a function to split the column names into components
def split_column_name(column_name):
    # Use regular expression to split the column name by underscores
    components = re.split('_+', column_name)
    return components

# Define a function to match components with abbreviations
def match_abbreviations(components):
    full_forms = []  # Using set to ensure deduplication
    for component in components:
        # Filter abbreviations based on component length
        possible_abbreviations = abbreviations_df[(abbreviations_df['Abbreviation'].str.len() == len(component)) & (len(component) >= 2)]  # Adjust the threshold as needed
        matches = process.extractOne(component, possible_abbreviations['Abbreviation'], scorer=fuzz.partial_ratio)
        if matches and matches[1] >= 98 and len(matches[0]) >= len(component):  # Adjust the thresholds as needed
            full_forms.append((component, abbreviations_df.loc[abbreviations_df['Abbreviation'] == matches[0], 'Full_Form'].values[0]))
    return list(set(full_forms))

# Initialize an empty set to collect abbreviation components and their full forms
abbreviation_set = []

display(tables[tables['columns'].str.contains("RA")])
# Iterate over columns
for columns in tables[tables['columns'].str.contains("RA")].head(10)['columns']:
    for column in remove_consecutive_duplicates(columns.strip().lstrip('[').rstrip(']').split(', ')):
        components = split_column_name(column)
        full_forms = match_abbreviations(components)
        abbreviation_set.extend(full_forms)
        print(column, full_forms)
print(list(set(abbreviation_set)))

Abbreviation                     Full_Form
0            RES                   Reservation
1             RA              Rental Agreement
2           DISC                      Discount
3            LOC                      Location
4          LYLTY                       Loyalty
..           ...                           ...
99           HLE           Hertz Local Edition
100          WWD                    World Wide
101          DAC  Direct Access Hold Indicator
102           SF                    Salesforce
103          OBJ                        Object

[104 rows x 2 columns]

table_catalog        table_schema          table_name  \
0         datalabs   lab_sfmc_extracts         conversions   
1         datalabs   lab_sfmc_extracts               lists   
2         datalabs   lab_sfmc_extracts            sendjobs   
3         datalabs   lab_sfmc_extracts                sent   
4         datalabs   lab_sfmc_extracts        statuschange   
...            ...                 ...                 ...   
5010  teradata_gdw  information_schema  routine_privileges   
5011  teradata_gdw  information_schema            routines   
5012  teradata_gdw  information_schema    table_privileges   
5013  teradata_gdw  information_schema   volume_privileges   
5014  teradata_gdw  information_schema             volumes   

                                                columns  
0     [ClientID, SendID, SubscriberKey, EmailAddress...  
1     [ClientID, ListID, Name, Description, DateCrea...  
2     [ClientID, SendID, FromName, FromEmail, SchedT...  
3     [ClientID, SendID, SubscriberKey, EmailAddress...  
4     [ClientID, SubscriberKey, EmailAddress, Subscr...  
...                                                 ...  
5010  [grantor, grantor, grantee, grantee, specific_...  
5011  [specific_catalog, specific_catalog, specific_...  
5012  [grantor, grantor, grantee, grantee, table_cat...  
5013  [grantor, grantor, grantee, grantee, volume_ca...  
5014  [volume_catalog, volume_catalog, volume_schema...  

[5015 rows x 4 columns]

table_catalog table_schema                        table_name  \
5     fleet_prd1_us    analytics                sales_full_history   
22              hdp   hdp_bronze              dash_historical_ramc   
33              hdp   hdp_bronze               eam_r5actchecklists   
34              hdp   hdp_bronze                  eam_r5activities   
37              hdp   hdp_bronze                     eam_r5objects   
...             ...          ...                               ...   
4987         rmp_na     rda_gold  cms_tier_recommendations_history   
4988         rmp_na     rda_gold                       hfs_geo_loc   
4989         rmp_na     rda_gold                      mdm_location   
4990         rmp_na     rda_gold                     ra_attributes   
4998         rmp_na     rda_gold                 vehicle_hierarchy   

                                                columns  
5     [VIN, VIN, REAL_VIN, REAL_VIN, ACTIVE, ACTIVE,...  
22    [MC-ARRAY, MC-ARRAY, RAMC_RA_PART, RAMC_RA_PAR...  
33    [ACK_ACT, ACK_ACT, ACK_CODE, ACK_CODE, ACK_COM...  
34    [ACT_ACT, ACT_ACT, ACT_ASSIGNMENTSTATUS, ACT_A...  
37    [OBJ_ACCESSIBLE, OBJ_ACCESSIBLE, OBJ_ACD, OBJ_...  
...                                                 ...  
4987  [CPD, CPD, SUPER_SEGMENT, SUPER_SEGMENT, BRAND...  
4988  [SOURCE, SOURCE, AREA, AREA, LOC, LOC, VAW_LOC...  
4989  [ROWID_OBJECT, ROWID_OBJECT, COUNTRY_CODE_ALPH...  
4990  [RA_NUM, RA_NUM, RESV_NUM, RESV_NUM, CHKOUT_DT...  
4998  [SIPP, SIPP, HERTZ_CAR_CLASS, HERTZ_CAR_CLASS,...  

[517 rows x 4 columns]

VIN []
REAL_VIN []
ACTIVE []
SOLD_MONTH []
SOLD_DATE []
OWN_AREA []
UNIT []
OWN_COUNTRY []
OWN_TYPE []
SERVICE_UNIT []
BROKEN_IND [('IND', 'Indicator')]
VDA []
CONDITION []
INVENTORY_DATE []
STATUS []
SOURCE []
CHANNEL []
PL_REASON []
ADD_DATE []
DAYS_ACTIVE_SINCE_SALE []
SOLD_AGE []
MILEAGE []
CAP_COST []
DAILY_BV []
SOLD_PRICE []
ACCOUNTING_PL []
CURRENT_DEP_AMT []
LAST_RENT_DATE []
LIFE_DEP_RATE []
LIFE_DEP_RATE_PERCENT []
CAR_CLASS []
VEHICLE_CLASS_NAME []
RALLY_CAR_GROUP []
MODEL_YEAR []
OEM []
MAKE []
MODEL_GROUP []
STYLE_ID [('ID', 'Identifier')]
MODEL []
TRIM []
COLOR []
GENERIC_COLOR []
ENGINE []
DRIVETRAIN []
LAST_RENT_AREA []
LAST_RENT_LOC [('LOC', 'Location')]
DMS_HERTZ_LOC [('LOC', 'Location')]
DMS_ORDER_LOC [('LOC', 'Location')]
DMS_BOS []
DMS_DEALER []
DMS_DEALER_STATE []
DMS_REP []
DMS_REP_REGION []
DMS_FUNDED_DATE_TIME []
DMS_COMBO []
IMS_AASC_ID [('ID', 'Identifier')]
IMS_WO []
IMS_FLOOR_REQUESTED []
IMS_FLOOR_ACCEPTED []
IMS_GRADE []
IMS_EST_DAMAGE []
IMS_BUYER_NAME 